In [1]:
import csv
import os
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from pandas.tslib import Timestamp

# Spelling corrector (based on http://norvig.com/spell-correct.html)

In [2]:
def P(word):
    "Probability of `word`."
    N = sum(WORDS.values())
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

# def candidates(word):
#     "Generate possible spelling corrections for word."
#     return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnñopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# def edits2(word):
#     "All edits that are two edits away from `word`."
#     return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Ciudades y barrios

## Socios activos 2016

In [3]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [4]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv('socios_activos_2016.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 399 ms, sys: 91 µs, total: 399 ms
Wall time: 397 ms


In [5]:
perfiles_df.shape

(50272, 11)

In [6]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota colombia,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota colombia,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota colombia,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro colombia,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,la soledad,bogota colombia,CEM,,,,,


## Ciudades, población y departamentos (Wikipedia)

In [7]:
dtypes = {
    'Ciudad': str,
    'Población': np.int32,
    'Departamento': str,
}

ciudades_df = pd.read_csv(
    'ciudades_colombia.csv',
    sep='\t',
    header=-1,
    names=['Wiki index', 'Ciudad', 'Población', 'Departamento'],
    usecols=['Ciudad', 'Población', 'Departamento'],
    dtype=dtypes,
    thousands='.',
)

In [8]:
ciudades_df.head()

,Ciudad,Población,Departamento
0,Bogotá,7840116,Cundinamarca
1,Medellín,2214494,Antioquia
2,Cali,2119908,Valle del Cauca
3,Barranquilla,1146359,Atlántico
4,Cartagena,892545,Bolívar


In [9]:
%%time

def process_field_ciudad(x):
    r = x['Ciudad'].strip().lower()
    r = r.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    return r

ciudades_df['Ciudad'] = ciudades_df.apply(process_field_ciudad, axis=1)

CPU times: user 27.5 ms, sys: 0 ns, total: 27.5 ms
Wall time: 26.8 ms


In [10]:
%%time

def process_field_departamento(x):
    r = x['Departamento'].strip().lower()
    r = r.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    return r

ciudades_df['Departamento'] = ciudades_df.apply(process_field_departamento, axis=1)

CPU times: user 35.4 ms, sys: 0 ns, total: 35.4 ms
Wall time: 34.5 ms


In [11]:
ciudades_df.head()

,Ciudad,Población,Departamento
0,bogota,7840116,cundinamarca
1,medellin,2214494,antioquia
2,cali,2119908,valle del cauca
3,barranquilla,1146359,atlantico
4,cartagena,892545,bolivar


In [12]:
ciudades_counter = Counter(dict((x['Ciudad'], x['Población']) for i, x in ciudades_df.iterrows()))

In [13]:
departamentos_df = ciudades_df.groupby('Departamento').sum()
departamentos_counter = Counter(dict((i, x['Población']) for i, x in departamentos_df.iterrows()))
departamentos_counter['colombia'] = departamentos_df['Población'].sum()

In [14]:
common = set(ciudades_counter).intersection(departamentos_counter)

In [15]:
common

{'arauca',
 'bolivar',
 'boyaca',
 'caldas',
 'colombia',
 'cordoba',
 'nariño',
 'risaralda',
 'sucre'}

# Limpiando datos

## Ciudades

In [16]:
perfiles_df['Ciudad'].nunique()

1327

In [17]:
perfiles_df['Ciudad'].value_counts().head()

bogota colombia    18523
bogota              4384
tunja colombia      2921
ibague colombia     2179
neiva colombia      1744
Name: Ciudad, dtype: int64

### Eliminando signos de puntuación

In [18]:
%%time

def delete_punctuation(x):
    r = x['Ciudad']
    for p in punctuation:
        r = r.replace(p, ' ')
    r = ' '.join(r.split())
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(delete_punctuation, axis=1)

CPU times: user 1.82 s, sys: 4.11 ms, total: 1.83 s
Wall time: 1.83 s


In [19]:
perfiles_df['Ciudad'].nunique()

1146

### Corregiendo terminaciones

In [20]:
endings = []
for x in perfiles_df['Ciudad']:
    y = x.split()
    if len(y) > 1:
        z = y[-1]
        endings.append(z)
endings_counter = Counter(endings)

In [21]:
endings_counter.most_common(50)

[('colombia', 40101),
 ('col', 2525),
 ('cundinamarca', 163),
 ('amazonas', 67),
 ('tolima', 52),
 ('atlantico', 33),
 ('colobia', 31),
 ('quindio', 27),
 ('cauca', 27),
 ('brasil', 21),
 ('caldas', 19),
 ('bogota', 18),
 ('coombia', 17),
 ('santander', 16),
 ('colombi', 16),
 ('isla', 16),
 ('andres', 16),
 ('huila', 14),
 ('antioquia', 13),
 ('marta', 13),
 ('colomia', 13),
 ('cordoba', 12),
 ('clombia', 11),
 ('sucre', 11),
 ('colomnbia', 11),
 ('bolivar', 10),
 ('colomiba', 10),
 ('olombia', 9),
 ('colmbia', 8),
 ('bia', 8),
 ('choco', 8),
 ('coolombia', 8),
 ('cund', 7),
 ('c', 7),
 ('nariño', 6),
 ('calera', 6),
 ('valle', 6),
 ('cundinamrca', 6),
 ('islas', 5),
 ('colomba', 5),
 ('colomvbia', 5),
 ('boyaca', 5),
 ('cololombia', 5),
 ('coilombia', 4),
 ('cololmbia', 4),
 ('colombias', 4),
 ('colommbia', 4),
 ('colombnia', 4),
 ('colombbia', 4),
 ('rosario', 3)]

In [22]:
%%time

def correct_endings(x):
    r = x['Ciudad']
    rl = r.split()
    while len(rl) > 1 and (rl[-1] in departamentos_counter or rl[-1] in ['col', 'isla', 'cund', 'islas']):
        rl = rl[:-1]
    r = ' '.join(rl)
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(correct_endings, axis=1)

CPU times: user 1.54 s, sys: 3.45 ms, total: 1.55 s
Wall time: 1.54 s


In [23]:
perfiles_df['Ciudad'].nunique()

896

### Corregiendo abreviaciones

In [24]:
%%time

def correct_abrevs(x):
    c1 = x['Ciudad']
    r = c1
    if c1 not in ciudades_counter:
        candidates = set()
        if len(c1) > 1 and len(c1.split()) == 1:
            for c2 in ciudades_counter:
                if len(c2.split()) == 1  and c2 != c1 and c2.startswith(c1):
                    candidates.add(c2)
            if candidates:
                r = max([(c, ciudades_counter[c]) for c in candidates], key=lambda x: x[1])[0]
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(correct_abrevs, axis=1)

CPU times: user 3.23 s, sys: 0 ns, total: 3.23 s
Wall time: 3.23 s


In [25]:
perfiles_df['Ciudad'].nunique()

878

### Corregiendo nombres de ciudades

In [26]:
WORDS = None

In [27]:
correcciones_aux = {}
for c in ciudades_counter:
    correcciones_aux[(c, 'ciudad')] = c
for c in departamentos_counter:
    correcciones_aux[(c, 'departamento')] = c
    for w in c.split():
        correcciones_aux[(w, 'departamento')] = w

In [28]:
ciudad_dept_df = ciudades_df.set_index('Ciudad')['Departamento']

ciudades_departamentos_dict = defaultdict(set)
for c, d in ciudad_dept_df.iteritems():
    ciudades_departamentos_dict[c].add(d)
ciudades_departamentos_dict = dict(ciudades_departamentos_dict)

def check_departamento(x, y):
    if x in ciudades_departamentos_dict and y in ciudades_departamentos_dict and \
       not ciudades_departamentos_dict[x].intersection(ciudades_departamentos_dict[y]):
        r = x
    else:
        r = y
    return r

In [29]:
%%time

def my_correction_1(x, tag):
    global WORDS
    if (x, tag) not in correcciones_aux:
        if tag == 'ciudad':
            WORDS = ciudades_counter
        else: # tag == 'departamento'
            WORDS = departamentos_counter
        correcciones_aux[(x, tag)] = correction(x)
    return correcciones_aux[(x, tag)]

def correct_ciudad(x):
    ciudad = x['Ciudad']
    if not ciudad:
        r = ciudad
    elif ciudad in ciudades_counter:
        r = ciudad
    elif my_correction_1(ciudad, 'ciudad') in ciudades_counter:
        aux = my_correction_1(ciudad, 'ciudad')
        r = check_departamento(ciudad, aux)
    else:
        words = ciudad.split()
        if len(words) == 1:
            r = ciudad
        else: # len(words) >= 2
            ok = []
            for w in words:
                if not (w in departamentos_counter or my_correction_1(w, 'departamento') in departamentos_counter):
                    ok.append(w)
            aux = ' '.join(ok)
            if aux in ciudades_counter:
                r = aux
            else:
                aux = my_correction_1(aux, 'ciudad')
                r = check_departamento(ciudad, aux)
    return r

perfiles_df['Ciudad'] = perfiles_df.apply(correct_ciudad, axis=1)

CPU times: user 1.77 s, sys: 3.91 ms, total: 1.77 s
Wall time: 1.77 s


In [30]:
perfiles_df['Ciudad'].nunique()

524

## Barrios

In [31]:
perfiles_df['Barrio'].nunique()

11092

In [32]:
perfiles_df['Barrio'].value_counts().head()

centro       1153
chapinero     430
              361
palermo       265
fontibon      242
Name: Barrio, dtype: int64

### Eliminando signos de puntuación

In [33]:
%%time

def delete_punctuation(x):
    r = x['Barrio']
    for p in punctuation:
        r = r.replace(p, ' ')
    r = ' '.join(r.split())
    return r

perfiles_df['Barrio'] = perfiles_df.apply(delete_punctuation, axis=1)

CPU times: user 1.94 s, sys: 77 µs, total: 1.94 s
Wall time: 1.93 s


In [34]:
perfiles_df['Barrio'].nunique()

10529

### Eliminando otros caracteres

In [35]:
chars = set()

for x in perfiles_df.groupby('Barrio').count().index:
    y = x.replace(' ', '')
    for z in y:
        if not (z in '0123456789' or z in 'abcdefghijklmnñopqrstuvwxyz' or z in ' '):
            chars.add(z)

In [36]:
chars

{'°', 'º', '¿', 'à', 'è', 'ì', 'ò', 'ö', 'ù', 'ü'}

In [37]:
chars_corrections = {
    '°': ' ',
    'º': '',
    '¿': '',
    'à': 'a',
    'è': 'e',
    'ì': 'i',
    'ò': 'o',
    'ö': 'o',
    'ù': 'u',
    'ü': 'u'}

In [38]:
%%time

def delete_chars(x):
    r = x['Barrio']
    for p in chars_corrections:
        r = r.replace(p, chars_corrections[p])
    r = ' '.join(r.split())
    return r

perfiles_df['Barrio'] = perfiles_df.apply(delete_chars, axis=1)

CPU times: user 1.67 s, sys: 0 ns, total: 1.67 s
Wall time: 1.67 s


In [39]:
perfiles_df['Barrio'].nunique()

10512

### Normalizando nombres de barrios

In [40]:
palabras = []
for x in perfiles_df['Barrio']:
    palabras.extend(x.split())
palabras_counter = Counter(palabras)

In [41]:
for x, y in palabras_counter.most_common():
    if y > 7 and len(x) < 4 and x not in stopwords.words('spanish'):
        print(x,y)

san 3145
ii 486
sur 446
2 219
sol 210
1 199
i 185
20 154
ana 153
rio 151
paz 150
urb 125
iii 116
7 107
vi 106
av 106
fe 105
via 92
luz 91
sta 84
b 79
3 74
iv 69
bay 52
vda 52
12 49
4 49
rey 39
n 39
11 39
j 38
uvo 29
s 28
xii 27
c 26
5 26
eva 26
pio 26
don 24
km 21
6 19
d 19
9 19
15 18
pie 18
mar 18
8 17
ed 16
p 16
brr 15
xxi 15
xxx 15
23 15
dos 15
31 13
v 12
jj 11
ap 11
et 11
ix 10
13 10
ext 9
14 9
30 9
10 9
oro 8
16 8
29 8
pan 8
poa 8
br 8


In [42]:
norm_barrios = {
    'urb': 'urbanización',
    'av': 'avenida',
    'vda': 'vereda',
    'sta': 'santa',
    'brr': 'barrio',
    'br': 'barrio',
    'suc': 'sucurusal',
    'ur': 'urbanización',
    'u': 'urbanización',
}

In [43]:
%%time

def normalize_barrios(x):
    r = x['Barrio']
    rl1 = r.split()
    rl2 = []
    for p in rl1:
        if p not in norm_barrios:
            rl2.append(p)
        else:
            rl2.append(norm_barrios[p])
    r = ' '.join(rl2)
    return r

perfiles_df['Barrio'] = perfiles_df.apply(normalize_barrios, axis=1)

CPU times: user 1.57 s, sys: 0 ns, total: 1.57 s
Wall time: 1.57 s


In [44]:
perfiles_df['Barrio'].nunique()

10459

### Eliminando el nombre de la ciudad y departamento

In [45]:
%%time

words = ['avenida', 'barrio', 'calle', 'ciudad', 'ciudadela', 'corregimiento', 'hacienda', 'nuevo', 'nueva',
         'parque', 'paseo', 'punto', 'simon', 'sucursal', 'urbanizacion', 'vereda', 'via',
         'de', 'del', 'el', 'la', 'los', 'las']

def delete_nombre_ciudad_dept(x):
    b = x['Barrio']
    c = x['Ciudad']
    r = b
    if c:
        if c in ciudad_dept_df.index:
            dept = ciudad_dept_df.ix[c]
            if isinstance(dept, str):
                ds = [dept]
            else: # isinstance(dept, pd.core.series.Series)
                ds = list(dept)
            for d in ds:
                if b != d and b.endswith(' ' + d) and not any(b.endswith(x + ' ' + d) for x in words):
                    i = r.rfind(d)
                    r = r[:i-1]
        if b != c and b.endswith(' ' + c) and not any(b.endswith(x + ' ' + c) for x in words):
            i = r.rfind(c)
            r = r[:i-1]
    return r

perfiles_df['Barrio'] = perfiles_df.apply(delete_nombre_ciudad_dept, axis=1)

CPU times: user 6.86 s, sys: 0 ns, total: 6.86 s
Wall time: 6.86 s


In [46]:
perfiles_df['Barrio'].nunique()

10423

### Eliminando artículos

In [47]:
%%time

def delete_articulos(x):
    b = x['Barrio']
    r = b
    if b.startswith('el ') or b.startswith('la '):
        r = r[3:]
    elif b.startswith('los ') or b.startswith('las '):
        r = r[4:]
    return r

perfiles_df['Barrio'] = perfiles_df.apply(delete_articulos, axis=1)

CPU times: user 1.48 s, sys: 0 ns, total: 1.48 s
Wall time: 1.48 s


In [48]:
perfiles_df['Barrio'].nunique()

9911

### Eliminando el nombre de la ciudad en 'centro'

In [49]:
%%time

def delete_nombre_ciudad_centro(x):
    b = x['Barrio']
    r = b
    if 'centro' != b and 'centro' in b:
        aux = b.replace('centro', '').strip()
        if aux == x['Ciudad']:
            r = 'centro'
    return r

perfiles_df['Barrio'] = perfiles_df.apply(delete_nombre_ciudad_centro, axis=1)

CPU times: user 1.51 s, sys: 10 µs, total: 1.51 s
Wall time: 1.51 s


In [50]:
perfiles_df['Barrio'].nunique()

9903

### Corregiendo nombres de barrios

In [51]:
barrios_count_df = perfiles_df.groupby('Barrio')['Ciudad'].count()

In [52]:
barrios_counter = Counter(dict((x, y) for x, y in barrios_count_df.iteritems() if x and y > 10))

In [53]:
barrios_words = []
for x in barrios_counter:
    for y in x.split():
        barrios_words.append(y)

In [54]:
barrios_words_counter = Counter(barrios_words)

In [55]:
correcciones_aux = {}
for b in barrios_counter:
    correcciones_aux[(b, 'barrio')] = b
for b in barrios_words_counter:
    correcciones_aux[(b, 'b_palabra')] = b

In [56]:
barrios_ciudades_dict = defaultdict(set)
for i, x in perfiles_df.iterrows():
    b = x['Barrio']
    c = x['Ciudad']
    if b and c:
        barrios_ciudades_dict[b].add(c)
barrios_ciudades_dict = dict(barrios_ciudades_dict)

def check_ciudad(x, y):
    if x in barrios_ciudades_dict and y in barrios_ciudades_dict and \
       not barrios_ciudades_dict[x].intersection(barrios_ciudades_dict[y]):
        r = x
    else:
        r = y
    return r

In [57]:
%%time

def my_correction_2(x, tag):
    global WORDS
    if (x, tag) not in correcciones_aux:
        if tag == 'barrio':
            WORDS = barrios_counter
        else: # tag == 'b_palabra'
            WORDS = barrios_words_counter
        correcciones_aux[(x, tag)] = correction(x)
    return correcciones_aux[(x, tag)]

def correct_barrio(x):
    barrio = x['Barrio']
    if not barrio:
        r = barrio
    elif barrio in barrios_counter:
        r = barrio
    elif my_correction_2(barrio, 'barrio') in barrios_counter:
        aux = my_correction_2(barrio, 'barrio')
        r =  check_ciudad(barrio, aux)
    else:
        ok = []
        words = barrio.split()
        for w in words:
            ok.append(my_correction_2(w, 'b_palabra'))
        aux = ' '.join(ok)
        if aux in barrios_counter:
            r = check_ciudad(barrio, aux)
        else:
            r = barrio
    return r

perfiles_df['Barrio'] = perfiles_df.apply(correct_barrio, axis=1)

CPU times: user 6.86 s, sys: 28 ms, total: 6.88 s
Wall time: 6.88 s


In [58]:
perfiles_df['Barrio'].nunique()

9140

## Resultados finales

In [59]:
perfiles_df.shape

(50272, 11)

In [60]:
perfiles_df.to_csv('socios_activos_2016_ciudades_barrios.csv', index=False)